Pipeline for downstream analysis 
Compute phenotype distribution differences between resistant and sensitive patients 
Then from the significant condition - phenotype distribution identified earlier do a gene differently expressed analysis 

In [1]:
import pandas as pd

from functions.analysis import downstream_analysis
from functions.analysis_utils.MaBoSS_simulation.maboss_phenotype_patient import compute_phenotype_table, collect_group_data,  compute_mean_phenotype_values
from functions.analysis_utils.results_MaBoSS_visualization.boxplot_phenotype import create_boxplot
from functions.analysis_utils.genes_intervention.pers_interventions import tailor_bnd_genes_intervention
from functions.analysis_utils.stats.stats_proba import compute_mannwhitneyu_test_means


from functions.analysis_utils.results_MaBoSS_visualization.create_phenotypes_patients_table import (
    vizualise_table_phenotype_condition, plot_side_by_side_heatmaps
)

from functions.analysis_utils.gene_enrichment.genes_signature import create_results_gene_enrichment

from functions.analysis_utils.results_MaBoSS_visualization.patients_ids_phenotype_table import create_combined_table_patients

from functions.analysis_utils.stats.stats_proba import compute_power_calculation


In [2]:
# models inputs
inputs_list = ['EGF', 'FGF', 'TGFB', 'Androgen', 'Hypoxia', 'Nutrients', 'Carcinogen', 'Acidosis', 'TNF', 'SPOP']

# models outputs
phenotype_interest = ["Proliferation","Invasion","DNA_Repair","Migration","Apoptosis"]

drug_interest = 'Refametinib' 

folder_models = f"analysis/{drug_interest}/models"
folder_results = f"analysis/{drug_interest}/results"




In [3]:
# Load top patient IDs from files saved in pipeline 1
with open("analysis/Refametinib/top_resistant_ids.txt") as f:
    top_resistant_ids = [line.strip() for line in f if line.strip()]

with open("analysis/Refametinib/top_sensitive_ids.txt") as f:
    top_sensitive_ids = [line.strip() for line in f if line.strip()]

In [4]:
downstream_analysis(folder_results,drug_interest, top_resistant_ids, top_sensitive_ids, inputs_list, phenotype_interest
)

model_pers_bnd analysis/Refametinib/models/sensitive/pers_models/SIDM00337_Refametinib.bnd
model_pers_cfg analysis/Refametinib/models/sensitive/pers_models/SIDM00337_Refametinib.cfg


KeyboardInterrupt: 

In [ ]:
df_res_combined = pd.read_csv(f'analysis/{drug_interest}/results/resistant/combined_results.csv', index_col=0)
df_sens_combined = pd.read_csv(f'analysis/{drug_interest}/results/sensitive/combined_results.csv', index_col=0)

compute_power_calculation(df_res_combined, df_sens_combined)

/Users/romane/repos/icr/Attractor-Resistance/functions/analysis_utils/stats/stats_proba.py:266: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  nb_patients_required.loc[condition][phenotype] = sample_size


,Proliferation,Invasion,DNA_Repair,Migration,Apoptosis
EGF,18.791042,41.654398,103.509136,36.279665,98.768864
FGF,17.936395,45.133038,95.364734,23.910501,36.368998
TGFB,74.496187,12.78642,92.278753,20.926328,32.446122
Androgen,18.764184,41.732214,103.109088,36.305432,95.163919
Hypoxia,154.192599,45.056712,95.583822,27.429378,76.469441
Nutrients,13.337988,46.796746,99.700946,26.429946,78.827622
Carcinogen,27.461193,47.956598,38.397895,40.949117,23.061945
Acidosis,25.602784,48.896917,98.661664,17.179914,67.060411
TNF,166.335632,4.694297,91.578742,21.342457,102.47169
SPOP,13.21114,30.107543,88.131306,35.019088,207.711233


why two TNF in the boxplot ??
check the nb of patients required is okay